# Import package

In [2]:
#Install module
# %pip install --user --upgrade pandas

In [3]:
#Import package
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import requests

# import os
# from tqdm import tqdm
# from datetime import datetime
# import yfinance as yf
# import lxml
# from bs4 import BeautifulSoup #BeutifulSoup

# Some useful functions

In [4]:
#Get Alpha Vantage api key
def get_apikey(filename: str):
    with open(filename) as f:
        api_key = f.read().strip()
    f.close
    return api_key

In [5]:
#Get daily core stock data from Alpha Vantage from 2000-01 to now
def get_daily_stockdata(symbol:str, outputsize = 'compact', datatype = 'json'):
    function = 'TIME_SERIES_DAILY'
    datatype = datatype
    outputsize = outputsize
    ticker = symbol
    alpha_vantage_apikey = get_apikey(filename= 'dist/apikey_AlphaVantage')

    url = f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&outputsize={outputsize}&apikey={alpha_vantage_apikey}&datatype={datatype}'
    r = requests.get(url)
    df = pd.DataFrame.from_dict(r.json()['Time Series (Daily)'],orient='index')
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = df.rename_axis('Date').reset_index()
    df.insert(loc = 0, column = 'Ticker', value = ticker, allow_duplicates=True)
    return df

# Data Preparation

## Web Scrapping

This section will perform web scrapping to scrap all S&P500 tickers on wikipedia. https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

We can simply use padnas to get the table from wiki.

In [6]:
#Store the S&P information in pandas dataframe
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
tickers_df_list = pd.read_html(wiki_url)
tickers_df = tickers_df_list[0]
tickers_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [7]:
#Turn pandas dataframe to the list
tickers = tickers_df['Symbol'].values.tolist()
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'C

## Download the financial data

In [15]:
#Download the financial data from Alpha vantage
# tickers_test = ['AAPL','MMM','ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL']

stock_df = pd.DataFrame(columns=['Ticker','Date','Open', 'High', 'Low', 'Close', 'Volume'])
for ticker in tickers:
    try: stock_df = pd.concat([stock_df, get_daily_stockdata(ticker,outputsize='full')], ignore_index = True)
    except: continue

In [16]:
stock_df

,Ticker,Date,Open,High,Low,Close,Volume
0,MMM,2023-07-28,111.8700,112.4100,111.0450,111.8800,2910759
1,MMM,2023-07-27,110.4500,113.1400,110.4500,111.1900,5613488
2,MMM,2023-07-26,109.8400,113.0700,109.8400,112.6400,5594986
3,MMM,2023-07-25,107.5500,110.8500,107.0092,109.8300,8778880
4,MMM,2023-07-24,104.5300,105.6041,103.9800,104.2700,3333380
...,...,...,...,...,...,...,...
156951,WMB,1999-11-05,35.5300,36.0900,34.4800,34.7300,2316900
156952,WMB,1999-11-04,36.2200,36.5900,35.2900,35.6600,2580700
156953,WMB,1999-11-03,36.4600,36.7700,35.2900,35.6600,3178600
156954,WMB,1999-11-02,37.6400,37.7700,37.0800,37.3300,1821800
